In [1]:
from IPython.display import display, HTML
display(HTML("""<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{fontsize:12pt;padding:5px;}
table.dataframe{font-size:12px;}))
</style>
"""))

# <span style="color:red">ch09 10 Retrieval의 효율개선을 위한 전처리(table) </span>

# 벡터 DB : Chroma vs Pinecone

- Chroma : 인메모리 vector DB, 로컬메모리 vector DB
- Pinecone : 클라우드 vector DB (Pinecone console에 api key 생성 → .env (PINECONE_API_KEY등록))

# 1. Knowledge Base 구성을 위한 데이터 생성

In [4]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = Docx2txtLoader('./tax_docs/with_table.docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200
)
document_list = loader.load_and_split(text_splitter = text_splitter)

In [5]:
len(document_list)

225

In [6]:
# embedding : upstage embedding-query
# https://python.langchain.com/v0.2/docs/integrations/text_embedding/upstage

from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()
embedding = UpstageEmbeddings(
    model = "solar-embedding-1-large"
#     model = "embedding-query"
)

In [7]:
%%time

# pinecone vector database

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone()
# 데이터를 처음 업로드할 때
index_name = "tax-index-table"
database = PineconeVectorStore.from_documents(
    documents = document_list,
    embedding = embedding,
    index_name = index_name
)

# 업로드한 벡터DB 가져올 때
# database = PineconeVectorStore(
#     embedding = embedding,  # 질문을 임베딩하여 유사도 검색
#     index_name = index_name
# )

C:\Users\Admin\anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: total: 15.7 s
Wall time: 54.9 s


# 2. 제공되는 prompt를 활용하여 답변 생성

In [8]:
from langchain import hub
from langchain_openai import ChatOpenAI

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model="gpt-4.1-nano")

In [9]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(),
    chain_type_kwargs = {"prompt":prompt}
)

In [10]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
ai_message = qa_chain.invoke({'query':query})
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 약 1,608,000원입니다. 과세표준이 5,000만원 이하이므로 84만원에 (1,400만원 초과 금액의 15%)를 더하는 방식으로 계산하며, 실제 세액은 약 1,608,000원으로 추정됩니다. 정확한 금액은 공제 및 세율 적용에 따라 차이 날 수 있습니다.'}